In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
df_train = pd.read_csv("train.csv", delimiter = ",", encoding='latin1') # i needed to specify delimiter and encoding
df1_train = df_train[df_train["Protocol"] == "FTP"]

data_train = df1_train.to_numpy()

df_test = pd.read_csv("test.csv", delimiter = ",", encoding='latin1') # i needed to specify delimiter and encoding
df1_test = df_test[df_test["Protocol"] == "FTP"]

data_test = df1_test.to_numpy()

In [3]:
def add_folders(server, key):
    key_split = key.split("/")
    for i in range(0, len(key_split)):
        if "/".join(key_split[0:i]) not in server:
            server["/".join(key_split[0:i])] =  "d"

In [4]:
def analyze(server, data):
    current_path = "/"

    for ind in range(0, len(data)):
        row = data[ind,6]
        row_split = row.split(" ")

    #   a new user has login => set current path to default path
        if row_split[0] == "Response:" and row_split[1] == "230": 
            current_path = "/"

        if row_split[0] == "Request:" and row_split[1] == "CWD": 
            if(row_split[2] == "/"): current_path = "/"
            elif(ind + 1 < len(data)):
                response = data[ind + 1,6].split(" ")
                if(response[0] == "Response:" and response[1] == "250"):
                    result_path = current_path[1:].split("/")
                    add_path =  row_split[2].split("/")
                    while True:
                        if len(add_path) == 0 or add_path[0] != ".." : break
                        else :
                            add_path = add_path[1:]
                            result_path = result_path[:-1]
                    current_path = "/"
                    if(result_path != [""]):
                        current_path += "/".join(list(result_path))
                    if(add_path != [""]):
                        if(current_path != "/"): 
                            current_path += "/" 
                        current_path += "/".join(list(add_path))
            server[current_path] = "d"
            add_folders(server,current_path)


        if row_split[0] == "Request:" and row_split[1] == "CDUP":
            result_path = current_path.split("/")
            if(ind + 1 < len(data)):
                result_path = result_path[:-1]
                current_path = "/"
                if(result_path != [""]):
                    current_path += "/".join(list(result_path))



        if row_split[0] == "Request:" and row_split[1] == "DELE": 
            if(ind + 1 < len(data)):
                response = data[ind + 1,6].split(" ")
                if(response[0] == "Response:" and response[1] == "250"):
                    if(current_path == "/"):
                        a = server.pop(current_path + row_split[2], None)
                    else:
                        a = server.pop(current_path + "/" + row_split[2], None)

        if row_split[0] == "Request:" and row_split[1] == "MKD": 
            if(ind + 1 < len(data)):
                response = data[ind + 1,6].split(" ")
                if(response[0] == "Response:" and response[1] == "550" and response[-2] == "File" and response[-1]== "exists" ):
                    if(current_path == "/"):
                        server[current_path + row_split[2]] = "d"
                    else:
                        server[current_path + "/" + row_split[2]] = "d"
                if(response[0] == "Response:" and response[1] == "257"):
                    if(current_path == "/"):
                        server[current_path + row_split[2]] = "d"
                        add_folders(server,current_path + row_split[2])
                    else:
                        server[current_path + "/" + row_split[2]] = "d"
                        add_folders(server,current_path + "/" + row_split[2])

        if row_split[0] == "Request:" and row_split[1] == "RETR": 
            if(ind + 1 < len(data)):
                response = data[ind + 1,6].split(" ")
                if(response[0] == "Response:" and response[1] == "125"):
                    if(current_path == "/"):
                        server[current_path + row_split[2]] = "f"
                        add_folders(server,current_path + row_split[2])
                    else:
                        server[current_path + "/" + row_split[2]] = "f"
                        add_folders(server,current_path + "/" + row_split[2])




        if row_split[0] == "Request:" and row_split[1] == "RNFR": 
            if(ind + 3 < len(data)):
                response1 = data[ind + 1,6].split(" ")
                request1 = data[ind + 2,6].split(" ")
                response2 = data[ind + 3,6].split(" ")
                if(response1[0] == "Response:" and response1[1] == "350" and request1[0] == "Request:" and request1[1] == "RNTO"):
                    if(response2[0] == "Response:" and response2[1] == "250"):
                        if(current_path == "/"):
                            value = server.pop(current_path + row_split[2], None)
                            if(value is None): server[current_path + request1[2]] = "f"
                            else: server[current_path + request1[2]] = value
                            add_folders(server,current_path + request1[2])

                        else:
                            value = server.pop(current_path + "/" + row_split[2], None)
                            if(value is None): server[current_path + request1[2]] = "f"
                            else: server[current_path + request1[2]] = value
                            add_folders(server,current_path + request1[2])

                    #'''verifying that the file does not exsist in the server for sure'''
                    elif(response2[0] == "Response:" and response2[1] == "550"):
                        if(current_path == "/"):
                            a = server.pop(current_path + row_split[2], None)
                        else:
                            a = server.pop(current_path + "/" + row_split[2], None)

        if row_split[0] == "Request:" and row_split[1] == "STOR": 
            if(ind + 1 < len(data)):
                response = data[ind + 1,6].split(" ")
                if(response[0] == "Response:" and response[1] == "125"):
                    server[current_path + "/" + row_split[2]] = "f"
                    add_folders(server,current_path + "/" + row_split[2])


In [5]:
server = {}
analyze(server,data_train)

In [6]:
analyze(server,data_test)

In [7]:
server.pop("")
server.pop("/")

'd'

In [8]:
pd.Series(list(server.values()), index = list(server.keys())).to_csv("server.csv")

c:\users\m\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
